In [ ]:
#-*-coding:utf-8-*-
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# 머신러닝을 지원하는 sklearn 라이브러리 가져오기

# 평가 지표들을 활용할 수 있는 metrics 모듈을 불러오기
from sklearn import metrics
import tqdm
from sklearn.model_selection import train_test_split#학습 평가 데이터 분리
import seaborn as sns #matplot과 유사하게 시각화 라이브러리이지만 좀 더 깔끔함


# !pip install seaborn
# 손쉽게 레이블 인코딩을 하기 위해 레이블인코더를 임포트
from sklearn.preprocessing import LabelEncoder
#!pip install graphviz #시각화 외부라이브러리
from sklearn.tree import export_graphviz

# 빨간색 경고창 무시하기
import warnings
warnings.filterwarnings(action="ignore")
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.cluster import KMeans
#!pip install reload
# 진행과정
from tqdm import tqdm_notebook
from flask import session, escape
from flask import Flask

#sys.setdefaultencoding('utf-8')

In [ ]:
import sys
#reload(sys) 
#sys.setdefaultencoding('utf-8')
print(sys.getdefaultencoding())

In [ ]:
#!pip install flask-login

In [ ]:
#login_manager = LoginManager()

In [ ]:
wondu = pd.read_csv("../data/coco.csv")
wondu

In [ ]:
#중복데이터 제거!!
#wondu = wondu.drop_duplicates() 
#wondu

In [ ]:
wondu.columns

In [ ]:
# 각 컬럼 width 최대로
pd.set_option('max_colwidth', 100)
# wondu에서 두 컬럼중 첫번째 열만 뽑아줌
wondu[['원두이름','아로마']][:1]

In [ ]:
# 문자열(str, object)를 딕셔너리(dictionary)로 만드는 방법을 import
from ast import literal_eval

# dictionary 형태로 저장된 string 값을 literal_eval 을 통해 실제 dictionary 로 가공
# genres,keywords는 딕셔너리 형태의 string타입이기 떄문에 딕셔너리로 apply(가공)한것이다.
#wondu['원두이름'] = wondu['원두이름'].apply(literal_eval)
#wondu['아로마'] = wondu['아로마'].apply(literal_eval)

In [ ]:
wondu['아로마']

In [ ]:
#중복데이터 제거!!
wondu = wondu.drop_duplicates() 
#wondu

In [ ]:
# lambda: 버려도 되는 일시적인 함수 
# 함수가 생성된 곳에서만 필요합니다. 즉, 간단한 기능을 일반적인 함수와 같이 정의해두고 쓰는 것이 아니고 
#필요한 곳에서 즉시 사용하고 버릴 수 있습니다.
# wondu['아로마']여기가 딕셔너리 형태로 id, name 으로 형성이 되어 있었는데
#  apply lambda를 이용하여 리스트 내 여러 개의 딕셔너리의 'name' 키 찾아 리스트 객체로 변환.        
# 람다는 한번쓰고 버리니까 이걸 두번 실행하면 오류남 한번에 사용하셈

#wondu['원두이름'] = wondu['원두이름'].apply(lambda x : [ y['name'] for y in x])
#wondu['아로마'] = wondu['아로마'].apply(lambda x : [ y['name'] for y in x])
#wondu[['원두이름', '아로마']][:1]
#wondu['아로마']

In [ ]:
# 문서를 토큰 리스트로 변환한다.
# 각 문서에서 토큰의 출현 빈도를 센다.
# 각 문서를 BOW 인코딩 벡터로 변환한다.
# 토큰(단위를 뜻함)
# BOW 인코딩: 문서를 숫자 벡터로 변환하는 가장 기본적인 방법
# 이 과정을 다 해주는 걸 import해줌
#단어 들의 카운트(출현 빈도(frequency))로 여러 문서들을 벡터화.
#카운트 행렬, 단어 문서 행렬 (Term-Document Matrix, TDM))
#모두 소문자로 변환시키기 때문에 me 와 Me 는 모두 같은 특성이 된다.

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
# wondu['아로마']라는 컬럼에 wondu['아로마'] 에 문자단위가 구분되도록 ' 한칸을 띄어서 공백을 줌 '
# 그리고 '구분자'.join(리스트)
# '구분자'.join(리스트)를 이용하면 리스트의 값과 값 사이에 '구분자'에 들어온 구분자를 넣어서 하나의 문자열로 합쳐줌
#wondu['아로마향'] = wondu['아로마'].apply(lambda x : (' ').join(x))
# CountVectorizer 의 옵션은
#stop_words : 리스트 형태로 불용어로 처리하고자 하는 문자를 넣어주시면 됩니다.
#analyzer : 문자열 {‘word’, ‘char’, ‘char_wb’} 또는 함수로 어떤 단위로 토큰화 할지를 정의합니다.
#token_pattern : 토큰화 하는 정규표현식 패턴입니다.
#tokenizer : 사이킷런에서 기본으로 제공하고 있는 토큰화 방법이 아닌 커스텀한 함수로 토큰화 
#하고자 할 때 함수를 만들어서 넣어줄 수 있습니다.
#ngram_range : n-그램 범위로 단어를 몇 개로 토큰화 할지를 의미합니다.
#max_df : 문서에서 등장하는 최대 빈도수를 의미합니다.
#min_df : 문서에서 등장하는 최소 빈도수를 의미합니다.
# 그래서 fit_transform은 데이터에 맞춘 다음 변환되어 단어가 문자열로 변환된 것을 fit_transform으로 학습을 시켜준다.
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
wondu_mat = count_vect.fit_transform(wondu['아로마'])
print(wondu_mat.shape)


In [ ]:
count_vect.vocabulary_

In [ ]:
test = wondu_mat.toarray()
test

In [ ]:
vocal = count_vect.vocabulary_
vocal

In [ ]:
sorted_col = np.array(sorted(list(zip(list(vocal.values()),list(vocal.keys())))))[:,1]
sorted_col

In [ ]:
df = pd.DataFrame(test,columns=sorted_col)

In [ ]:
df

In [ ]:
ro = pd.get_dummies(wondu['로스팅'])
ro.reset_index()

In [ ]:
result = pd.concat([df,ro.reset_index(),wondu[['산도','단맛','쓴맛','바디감']].reset_index()],axis=1)
result.drop('index',axis=1,inplace=True) # 원두번호 삭제해버릴꺼지롱!!
result

In [ ]:
#X와 Y의 샘플 간의 코사인 유사성을 계산해주는거 import
#코사인 유사성 또는 코사인 커널은 유사성을 X와 Y의 정규화 된 내적으로 계산됨.
#K (X, Y) = <X, Y> / (|| X || * || Y ||)
#단어들이나 문장들을 벡터로 표현이 가능할 경우 이 벡터간의 거리를 
#이용해서 유사성을 알 수 있으며 본 포스팅은 대표적으로 많이 활용하는 코사인 유사도임
#wondu_mat학습해준 변수를 코사인 안에 넣어줌
# 같은 컬럼을 두개를 넣었기 때문에 shape이 같이 나오지 않았을까?
#wondu의 아로마 컬럼을 문자열 변환하고 피처 벡터화한 행렬 
#wondu_mat를 cosine_similarities()로 계산된 wondu_sim 객체는 
#wondu DataFrame의 행별 장르 유사도 값을 가지고 있음
from sklearn.metrics.pairwise import cosine_similarity
wondu_sim = cosine_similarity(result, result)
print(wondu_sim.shape)
print(wondu_sim[:2])

In [ ]:
cosine_similarity(result, result)   # 여기서 높을걸 가지고 하셈  cosine_similarity(wondu_mat[3], wondu_mat)

In [ ]:
wondu_sim

In [ ]:
wondu_sim.argsort()     

In [ ]:
# [:, ::-1] axis = 1 기준으로 2차원 numpy 배열 뒤집음
#코사인 유사도가 측정된 wondu_sim 객체를 Numpy의 argsort() 함수를 통해 유사도가 높은 순 정렬
#또한, 각 레코드에서 맨 뒤에 있는 인덱스 값을 비교 행 위치로 뒤집어 
#결과 값을 보자면 첫 번째(0번) 레코드 의 경우 자기 자신 0번을 제외하면 16번 레코드가 가장 유사도가 높고, 
#그 다음이 1번 레코드, 가장 유사도가 낮은 레코드는 8번 레코드라는 뜻
wondu_sim_sorted_ind = wondu_sim.argsort()[:, ::-1]
print(wondu_sim_sorted_ind[:1])

In [ ]:
def find_sim_wondu(df, sorted_ind, title_wondu,rosting, top_n=10):
    # 여기에 매개변수로 우리가 넣을 데이터를 입력해주자
    
    # 인자로 입력된 wondu DataFrame에서 'title' 컬럼이 입력된 title_wondu 값인 DataFrame추출
    # 여기서 컬럼을 써주자
    title_coffee = df[(df['아로마'] == title_wondu) & (df['로스팅'] == rosting)]   
    # title_wondu을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 wondu_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
   
    title_index = title_coffee.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [ ]:
# 불리언 인덱싱을 활용한, 산도 , 시티로스팅, 견과류향 을 모두 충족한 데이터를 찾아보리기!!
result[(result['산도']==2) & (result['시티로스팅']==True) & (result['견과류향']==True)]#wondu[wondu['아로마']=="견과류향"]

In [ ]:
wondu.shape

In [ ]:
# 견과류이면서 시티로스팅 뽑아줘!!!
similar_wondus = find_sim_wondu(wondu, wondu_sim_sorted_ind,'견과류향','시티로스팅',1)#wondu = result
similar_wondus[['원두이름', '아로마','로스팅','산도','바디감']]

In [ ]:
wondu[['원두이름','아로마','랭킹','산도']].sort_values('랭킹', ascending=False)[:5]

In [ ]:
# C = wondu['랭킹'].mean()
# m = wondu['바디감'].quantile(0.6)
# print('C:',round(C,3), 'm:',round(m,3))

In [ ]:
# percentile = 0.6
# m = wondu['바디감'].quantile(percentile)
# C = wondu['랭킹'].mean()

# def wondu_vote_average(record):
#     v = record['바디감']
#     R = record['랭킹']
    
#     return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

# wondu['랭킹'] = wondu.apply(wondu_vote_average, axis=1)

In [ ]:
wondu[['원두이름','아로마','로스팅','바디감','랭킹']].sort_values('랭킹',
                                                        ascending=False)[:5]

In [ ]:
wondu[wondu['아로마'].str.contains('꽃향')]

In [ ]:
def find_sim_wondu(df, sorted_ind, title_wondu,rosting,san, top_n):
    #아로마, 로스팅, 바디감을 입력받을꺼야!!
    title_coffee = df[(df['아로마'].str.contains(title_wondu)) & (df['로스팅']== rosting) & (df['산도'] == san)] 
    display(title_coffee)
    
    title_index = title_coffee.index.values
    display(title_index)
    
    # top_n의 2배에 해당하는 아로마 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    display(similar_indexes)
    
    similar_indexes = similar_indexes.reshape(-1)
    display(similar_indexes)
    # 중복 원두 제외
    unique_indexes = np.unique(similar_indexes)
    display(unique_indexes)
    
    # 기준 원두 index는 제외
#     similar_indexes = similar_indexes[similar_indexes != title_index]
#     display(similar_indexes)
    
    similar_indexes = set(unique_indexes) - set(title_index)
    display(similar_indexes)

    # top_n의 2배에 해당하는 후보군에서 랭킹 높은 순으로 top_n 만큼 추출 
    return df.iloc[list(similar_indexes)].sort_values('랭킹', ascending=True)[:top_n]  #랭킹 순으로 뽑아줘!=>여기에 단맛 넣으면 단맛 기준으로 정렬

similar_wondu = find_sim_wondu(wondu, wondu_sim_sorted_ind, '견과류향','시티로스팅',2,3) # 견과류향, 시티로스팅 , 바디감(2)에 가까운걸 5개뽑아줘!!
coffee=similar_wondu[['원두이름']]
####향,로스팅,바디감으로 거르고 그 후에 랭킹으로 한번더 정렬
coffee

In [ ]:
# def find_sim_wondu(df, sorted_ind, title_wondu,rosting,san, top_n=10):
#     #아로마, 로스팅, 바디감을 입력받을꺼야!!
#     title_coffee = df[(df['아로마'].str.contains(title_wondu)) & (df['로스팅']== rosting) & (df['산도'] == san)] 
#     title_index = title_coffee.index.values
    
#     # top_n의 2배에 해당하는 아로마 유사성이 높은 index 추출 
#     similar_indexes = sorted_ind[title_index, :(top_n*2)]
#     similar_indexes = similar_indexes.reshape(-1)
# # 기준 원두 index는 제외
#     similar_indexes = similar_indexes[similar_indexes != title_index]
    
#     # top_n의 2배에 해당하는 후보군에서 랭킹 높은 순으로 top_n 만큼 추출 
#     return df.iloc[similar_indexes].sort_values('랭킹', ascending=True)[:top_n]  #랭킹 순으로 뽑아줘!=>여기에 단맛 넣으면 단맛 기준으로 정렬

# similar_wondu = find_sim_wondu(wondu, wondu_sim_sorted_ind, '과일향','시티로스팅',2,2) # 견과류향, 시티로스팅 , 바디감(2)에 가까운걸 5개뽑아줘!!
# coffee=similar_wondu[['원두이름']]
# ####향,로스팅,바디감으로 거르고 그 후에 랭킹으로 한번더 정렬
# coffee

In [ ]:
wondu

In [ ]:
wondu[(wondu['아로마'].str.contains('과일향')) & (wondu['로스팅']== '풀시티로스팅')] 

In [ ]:
# 불리언 인덱싱을 활용한, 산도 , 시티로스팅, 견과류향 을 모두 충족한 데이터를 찾아보리기!!
result[(result['산도']==2) & (result['시티로스팅']==True) & (result['견과류향']==True)]#wondu[wondu['아로마']=="견과류향"]
result[(result['과일향']==True) &(result['시티로스팅']==True)]

In [ ]:
# asdasdasd

In [ ]:
# from flask import Flask
# app = Flask(__name__)
# @app.route('/')
# def index():   
#     return coffee

# if __name__=="__main__":
#     app.run(host='127.0.0.1', port='5000')

In [ ]:
# from flask import Flask
# from flask import redirect

# app = Flask(__name__) 

# @app.route('/')
# def index():     
#     # JSP로 만들어진 웹 페이지에 데이터 전송
#     # GET방식으로 데이터 전달 시 쿼리스트링 기술 활용
#     #  -> ?name=value&name=value&...
#     url = 'http://localhost:8081/Cafe/AjaxCoffeeController?data=coffee'
#     return redirect(url)

# if __name__ == '__main__':
#     app.run(host='127.0.0.1', port='5003') 

In [ ]:
# from flask import Flask, jsonify

# app = Flask(__name__)

# @app.route('/wondu', methods=['GET','POST'])
# def wondu():
#     if request.method == 'POST':
#        data = {'wonduName' : coffee}
#        return jsonify(data)

# if __name__ == '__main__':
#     app.run(host='127.0.0.1', port='5000') 

In [ ]:
# similar_wondu = find_sim_wondu(wondu, wondu_sim_sorted_ind, '견과류향','시티로스팅',3,1)
# coffee=similar_wondu['원두이름']
similar_wondu = find_sim_wondu(wondu, wondu_sim_sorted_ind,'초콜릿향','시티로스팅',2,1) # 견과류향, 시티로스팅 , 바디감(2)에 가까운걸 5개뽑아줘!!
coffee=similar_wondu['원두이름']
coffee.values[0]
coffee

In [ ]:
from flask import Flask
from flask import request, redirect

app = Flask(__name__) 

@app.route('/', methods=['GET','POST']) #/wondu
def index():   #wondu  
       
    if request.method == 'GET':
        cf1 = str(request.args['chk_info1'])
        cf2 = str(request.args['chk_info2'])
        cf3 = int(request.args['chk_info3'])
       
        #df.iloc[list(similar_indexes)].sort_values('랭킹', ascending=True)[:top_n]
        similar_wondu = find_sim_wondu(wondu, wondu_sim_sorted_ind, cf1,cf2,cf3,3) # 견과류향, 시티로스팅 , 바디감(2)에 가까운걸 5개뽑아줘!!
        coffee=similar_wondu['원두이름']
        print(cf1, cf2, cf3)

        # result = model() = request.args['num1']
        
        
    else:
        cf1 = str(request.form['chk_info1'])
        cf2 = str(request.form['chk_info2'])
        cf3 = int(request.form['chk_info3'])
        print(cf1, cf2, cf3)
        #df.iloc[list(similar_indexes)].sort_values('랭킹', ascending=True)[:top_n]
        similar_wondu = find_sim_wondu(wondu, wondu_sim_sorted_ind, cf1,cf2,cf3,3) # 견과류향, 시티로스팅 , 바디감(2)에 가까운걸 5개뽑아줘!!
        coffee=similar_wondu['원두이름']
        #similar_wondu = find_sim_wondu(wondu, wondu_sim_sorted_ind, 'cf1','cf2',cf3,1) # 견과류향, 시티로스팅 , 바디감(2)에 가까운걸 5개뽑아줘!!
        #coffee=similar_wondu[['원두이름']]
       
    print(len(coffee.values))
    url = 'http://localhost:8081/Cafe/recocoffee.do?result='+coffee.values[1] #coffee.values[0]
    return redirect(url)

if __name__ == '__main__':
    app.config['JSON_AS_ASCII'] = False
    app.run(host='127.0.0.1', port='5003') 

In [ ]:
from flask import Flask
from flask import request, redirect
app = Flask(__name__) 
@app.route('/', methods=['GET','POST'])
def index():        
    if request.method == 'GET':
        cf1 = str(request.args['chk_info1'])
        cf2 = str(request.args['chk_info2'])
        cf3 = int(request.args['chk_info3'])       
        similar_wondu = find_sim_wondu(wondu, wondu_sim_sorted_ind, cf1,cf2,cf3,3) 
        coffee=similar_wondu['원두이름']
        print(cf1, cf2, cf3)                
    else:
        cf1 = str(request.form['chk_info1'])
        cf2 = str(request.form['chk_info2'])
        cf3 = int(request.form['chk_info3'])
        print(cf1, cf2, cf3)        
        similar_wondu = find_sim_wondu(wondu, wondu_sim_sorted_ind, cf1,cf2,cf3,3) 
        coffee=similar_wondu['원두이름']            
    print(len(coffee.values))
    url = 'http://localhost:8081/Cafe/recocoffee.do?result='+coffee.values[1] 
    return redirect(url)
if __name__ == '__main__':
    app.config['JSON_AS_ASCII'] = False
    app.run(host='127.0.0.1', port='5003') 

In [ ]:
# from flask import Flask
# from flask import request, redirect

# app = Flask(__name__) 

# @app.route('/wondu', methods=['GET','POST'])
# def wondu():     
        
#     if request.method == 'GET':
#         cf1 = str(request.form['chk_info1'])
#         cf2 = str(request.form['chk_info2'])
#         cf3 = int(request.form['chk_info3'])
#         #similar_wondu = find_sim_wondu(wondu, wondu_sim_sorted_ind, 'cf1','cf2',cf3,1) # 견과류향, 시티로스팅 , 바디감(2)에 가까운걸 5개뽑아줘!!
#         #coffee=similar_wondu[['원두이름']]
      
#        similar_wondu = find_sim_wondu(wondu, wondu_sim_sorted_ind, 'cf1','cf2',cf3,1) # 견과류향, 시티로스팅 , 바디감(2)에 가까운걸 5개뽑아줘!!
#        coffee=similar_wondu[['원두이름']]
#        ####향,로스팅,바디감으로 거르고 그 후에 랭킹으로 한번더 정렬

#         # result = model()
        
#     else:
#         cf1 = str(request.form['chk_info1'])
#         cf2 = str(request.form['chk_info2'])
#         cf3 = int(request.form['chk_info3'])
        
#         similar_wondu = find_sim_wondu(wondu, wondu_sim_sorted_ind, 'cf1','cf2',cf3,1)
#         coffee=similar_wondu[['원두이름']]
      
        
#         # result = model()

#     url = 'http://localhost:8081/Cafe/recodata.jsp?result='+coffee
#     #return redirect(url)
#     return jsonify(url)

# if __name__ == '__main__':
#     app.config['JSON_AS_ASCII'] = False
#     app.run(host='127.0.0.1', port='5003') 

In [ ]:
# from flask import Flask, jsonify

# app = Flask(__name__)

# @app.route('/location')
# def location():
#     data = {'대여소번호' : rent_num, '위도' : latitude2, '경도' : longitude2}
#     return jsonify(data)

# @app.route('/rankData')
# def rankData():
#     data2 = {'자치구': borough, '그룹번호' : group_num, '대여소명' : rent_name,
#              '위도' : latitude, '경도' : longitude,
#              '평균거리' : distance_mean, '총이용건수' : using, '가중평균 랭크' : weighted_rank}
#     return jsonify(data2)

# if __name__ == '__main__':
#     app.config['JSON_AS_ASCII'] = False
#     app.run(host='127.0.0.1', port='5000')

In [ ]:
# from flask import Flask

# app = Flask(__name__) # flask 서버객체 생성
 
    
# # route(경로): 외부에서 접근할 때 페이지 구분을 위한 경로지정
# @app.route('/')   # 5000 port 뒤에 / 를 사용하여 경로설정(설정된 경로의 페이지에 함수가 적용됨)
# def index():
#     return coffee # return을 하게 될 경우 사용자에게 전달


# # main함수의 시작을 의미
# # 비유하자면 자바의 main(String[] args)메소드와 같은 역할
# if __name__ == '__main__':
#     # 웹서버주소(host)(=IP주소)와 포트번호(port)(고정값)를 지정하여 웹 서버 실행
#     app.run(host='127.0.0.1', port='5000') 

In [ ]:
# from flask import Flask
# from flask import request, redirect

# app = Flask(__name__) 

# @app.route('/', methods=['GET','POST'])
# def index():     
       
#     if request.method == 'GET':
# #         num1 = int(request.args['num1'])
# #         num2 = int(request.args['num2'])        
#         # result = model()
#         coffee
#     else:
# #         num1 = int(request.form['num1'])
# #         num2 = int(request.form['num2'])
#         coffee
#         # result = model()

#     url = 'http://localhost:8081/Cafe/recommend.jsp'
#     return redirect(url)

# if __name__ == '__main__':
#     app.run(host='127.0.0.1', port='5002') 

In [ ]:
# from flask import Flask
# from flask import redirect

# app = Flask(__name__) 

# @app.route('/test')
# def index():     
#     # JSP로 만들어진 웹 페이지에 데이터 전송
#     # GET방식으로 데이터 전달 시 쿼리스트링 기술 활용
#     #  -> ?name=value&name=value&...
#     url = 'http://localhost:8081/Cafe/FrontContoller'
#     return redirect(url)

# if __name__ == '__main__':
#     app.run(host='127.0.0.1', port='5000') 

In [ ]:
# # 2. 비계층적 군집 분석 model
# model = KMeans(n_clusters=3, random_state=0, algorithm='auto')
# # n_clusters=3 : 군집의 개수 (k) (이미 알고 있음)
# # random_state=0 : seed 역할 (모델을 일정하게 생성 = 랜덤X)
# model.fit(result)

In [ ]:
# pre = model.predict(result)
# pre
# len(pre)

In [ ]:
# import matplotlib.pyplot as plt # 시각화
# plt.scatter(x=wondu['아로마'], y=wondu['로스팅'], c=pre)

In [ ]:
# clusters = linkage(y=result, method='complete', metric='euclidean')
# clusters
# clusters.shape

In [ ]:
# # 4. 덴드로그램 시각화 : 군집수 결정
# import matplotlib.pyplot as plt
# plt.figure( figsize = (25, 10) )
# dendrogram(clusters, leaf_rotation=90, leaf_font_size=12,)
# # leaf_rotation=90 : 글자 각도
# # leaf_font_size=20 : 글자 사이즈
# plt.show() 

In [ ]:

# # 5. 클러스터링(군집) 결과
# from scipy.cluster.hierarchy import fcluster # 지정한 클러스터 자르기

In [ ]:
# cut_tree = fcluster(clusters, t=3, criterion='distance')
# cut_tree # prediction

In [ ]:
# # irisDF에 군집 예측치 추가
# result.head()
# result['cluster'] = cut_tree
# result.head()

In [ ]:
# cluster_g = result.groupby('cluster')
# cluster_g.mean()